# Investment Decision
## Author: Snigdhayan Mahanta

`Monte Carlo Simulation` can be a powerful technique to get a quantitive idea about future outcomes based on simulation. I tried out the `MonteCarlo` package of `R` to assess its efficacy. Suppose one has 10000 EUR to invest in a project. There are two options:
1. Option 1 - low rate of return and low risk of failure
2. Option 2 - high rate of return and high risk of failure

I used `Monte Carlo Simulation` using some fictitious values for the above parameters to compute the average returns for the two options.

In [1]:
library(MonteCarlo, quietly = TRUE)

In [2]:
# Define a function that calculates the ROI on a risky investment, e.g., investment on a startup
ROI <- function(rate, risk, investment){
  # generate investment outcome:
  outcome <- sample(0:1, size=1, prob=c((risk/100), 1-(risk/100))) # 0 = failure, 1 = success
  
  # calculate ROI:
  result <- ((rate/100)*investment*outcome)
  
  # return result:
  return(list("ROI" = result))
}

In [3]:
# define parameter grid:
investment <- 10000 # 10k EUR to invest

# Rate of return as percentage
low_rate <- 400 # 400% rate of return
high_rate <- 800 # 800% rate of return

# Risk of failure as percentage
low_risk <- 60 # 60% risk of failure
high_risk <- 80 # 80% risk of failure

rate <- c(low_rate, high_rate)
risk <- c(low_risk, high_risk)

In [4]:
# collect parameter grids in list:
param_list=list("rate" = rate, "risk" = risk, "investment" = investment)

In [5]:
# run simulation:
repetitions <- 1000
MC_result<-MonteCarlo(func = ROI, nrep = repetitions, param_list = param_list)
summary(MC_result)

Grid of  4  parameter constellations to be evaluated. 
 
Progress: 
 
  |======================================================================| 100%
 
Simulation of function: 

function(rate, risk, investment){
  # generate investment outcome:
  outcome <- sample(0:1, size=1, prob=c((risk/100), 1-(risk/100))) # 0 = failure, 1 = success
  
  # calculate ROI:
  result <- ((rate/100)*investment*outcome)
  
  # return result:
  return(list("ROI" = result))
}
<bytecode: 0x000000001f80cd50>

Required time: 0.12 secs for nrep = 1000  repetitions on 1 CPUs 

Parameter grid: 

       rate : 400 800 
       risk : 60 80 
 investment : 10000 

 
1 output arrays of dimensions: 2 2 1 1000

In [6]:
# Aggregate results
x1 <- numeric(repetitions) # low rate, low risk
x2 <- numeric(repetitions) # high rate, high risk

for (i in c(1:repetitions)) {
    x1[i] <- MC_result$results$ROI[4*(i-1)+1]
    x2[i] <- MC_result$results$ROI[4*(i-1)+4]
}

In [7]:
# Average return - low rate and low risk
mean(x1)

[1] 15920

In [8]:
# Average return - high rate and high risk
mean(x2)

[1] 16640

One can easily change the simulation parameters and analyze different scenarios.